Necessary imports

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import numpy as np
import pandas as pd
import joblib
from sklearn.preprocessing import StandardScaler
from IPython.display import clear_output

# Check if GPU is available and set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Print device
if device.type == 'cuda':
    print('Using GPU for training')
else:
    print('Using CPU for training')

Using GPU for training


Configuring the model

In [4]:
# LJ Dataset class
class LJDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.X = self.data[['Density', 'PotEng', 'Enthalpy']].values.astype(np.float32)
        self.y = self.data[['Epsilon', 'Sigma']].values.astype(np.float32)
        self.len = len(self.data)

    def __len__(self):
        return self.len

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

# Neural Network Model for LJ parameters
class LJNet(nn.Module):
    def __init__(self):
        super(LJNet, self).__init__()
        self.fc1 = nn.Linear(3, 64)
        self.fc2 = nn.Linear(64, 64)
        self.fc3 = nn.Linear(64, 2)  # 2 output nodes for Epsilon and Sigma

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

Training

In [2]:


# Load the LJ dataset
lj_dataset = LJDataset('Data/simulation_results.csv')
train_loader = DataLoader(lj_dataset, batch_size=32, shuffle=True)

# Normalize the features using StandardScaler and save the scaler
scaler = StandardScaler()
lj_dataset.X = scaler.fit_transform(lj_dataset.X)

# Save the scaler
joblib.dump(scaler, 'Models/DL/scaler.pkl')

# Initialize the LJ model, loss function, and optimizer
lj_model = LJNet().to(device)
criterion = nn.MSELoss()
optimizer = optim.Adam(lj_model.parameters(), lr=0.001)


total_epochs = 1000

for epoch in range(1, total_epochs + 1):  
    for batch_idx, (inputs, targets) in enumerate(train_loader, 1):
        inputs, targets = inputs.to(device), targets.to(device)
        
        optimizer.zero_grad()
        outputs = lj_model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

    # Display progress
    clear_output()
    print(f"Current Iteration: {epoch}/{total_epochs}")

# Save the trained LJ model
torch.save(lj_model.state_dict(), 'Models/DL/lj_model.pth')


Current Iteration: 1000/1000


Loding the saved model

In [6]:
import warnings

warnings.simplefilter('ignore', UserWarning)

# Function to calculate percentage relative error
def percentage_relative_error(actual, predicted):
    # Convert inputs to PyTorch tensors if not already
    actual = torch.tensor(actual).to(device)
    predicted = torch.tensor(predicted).to(device)
    
    return torch.abs((predicted - actual) / actual) * 100

# Check if GPU is available and set device
device = torch.device('cuda')

# Load the trained LJ model
lj_model = LJNet()
lj_model.load_state_dict(torch.load('Models/DL/lj_model.pth'))
lj_model.to(device)
lj_model.eval()

# Load the scaler
scaler = joblib.load('Models/DL/scaler.pkl')   

# Define the custom input data for testing
custom_data = pd.DataFrame(np.array([[1.3052555611111112,-652.6723927777779,-499.243345]]), columns=['Density', 'PotEng', 'Enthalpy'])

# Normalize the custom input data using the loaded scaler
custom_data_normalized = scaler.transform(custom_data) 

# Convert to PyTorch tensor and move to the appropriate device
inputs = torch.tensor(custom_data_normalized, dtype=torch.float32).to(device)

# Make prediction
with torch.no_grad():
    outputs = lj_model(inputs)
predicted_epsilon, predicted_sigma = outputs.cpu().numpy()[0]  # Move outputs to CPU and convert to numpy array

# Define the real values (actual values)
real_epsilon = 0.2385
real_sigma = 3.405

# Print the results with specific formatting
print("Epsilon:")
print(f"real value: {real_epsilon}  predicted value: {predicted_epsilon:.4f} \nrelative error: {percentage_relative_error(real_epsilon, predicted_epsilon):.2f} %")

print("\nSigma:")
print(f"real value: {real_sigma}  predicted value: {predicted_sigma:.4f} \nrelative error: {percentage_relative_error(real_sigma, predicted_sigma):.2f} %")

Epsilon:
real value: 0.2385  predicted value: 0.2396 
relative error: 0.46 %

Sigma:
real value: 3.405  predicted value: 3.4315 
relative error: 0.78 %
